# Mappr

> Scale up evaluation report mapping against evaluation frameworks using agentic workflows


## Approach

**Problem**: Manually mapping evaluation reports against IOM's Strategic Results Framework (SRF) is time-consuming and resource-intensive with ~150 outputs to analyze.

**Solution**: Multi-stage pipeline leveraging Global Compact for Migration (GCM) as a pruning mechanism:

### Stage 1: Sequential Classification
1. **Enablers & Cross-cutting Priorities** (11 items) - Fast parallel analysis to identify meta-evaluation nature
2. **GCM Objectives** (23 items) - Informed by Stage 1 results, uses condensed representations for efficiency

### Stage 2: Targeted SRF Analysis  
- Use GCM results + `gcm_srf_lut` to prune ~150 SRF outputs to ~20-50 relevant ones
- Deep analysis with full hierarchy context (objective → outcome → output → indicators)

### Key Features
- **Agentic workflow**: LLM navigates document headings, explores sections iteratively until confident
- **DSPy signatures**: Structured reasoning with built-in tracing for evaluation
- **Rate-limited parallelization**: Respects API constraints (15 RPM) using fastcore
- **False positive bias**: Better to over-include than miss relevant mappings

In [ ]:
#| default_exp mappr

::: {.callout-warning}
This notebook is a work in progress.
:::

In [ ]:

#| exports
from pathlib import Path
from functools import reduce
from toolslm.md_hier import *
from rich import print
import json
from fastcore.all import *

from typing import List
import dspy


In [ ]:
#| exports
from dotenv import load_dotenv
import os

load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

In [ ]:
#| exports
cfg = AttrDict({
    'lm': 'gemini/gemini-2.0-flash-exp',
    'api_key': GEMINI_API_KEY,
    'max_tokens': 8192,
    'track_usage': False,
})

In [ ]:
#| eval: false
doc = Path("../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enriched")
pages = doc.ls(file_exts=".md").sorted(key=lambda p: int(p.stem.split('_')[1]))
report = '\n\n---\n\n'.join(page.read_text() for page in pages)
print(report[:1000])

# PPMi .... page 1

**Final Evaluation of the EU-IOM Joint Initiative for migrant protection and reintegration in the horn of Africa**

Final Evaluation Report, 17 March 2023

!(img-0.jpeg)

**EU-IOM** Joint Initiative for Migrant Protection and Reintegration

Project funded by the European Union
Project implemented by IOM

---

This Final Evaluation Report was commissioned by IOM and developed by the evaluation team of PPMI Group, including:
Loes van der Graaf, Rimantas Dumcius, Radvilė Bankauskaitė, Anna Kiss-Pal and Laura Daukšaitė, as well as by 
external expert Anthony Roger Plant. The evaluation team is grateful to all IOM staff and stakeholders to the 
JI-HoA for their time taken to participate in interviews. The team is especially grateful to the returnees, 
migrants, and community members who participated in Focus Group Discussions.

This publication was funded by the European Union. Its contents are the sole responsibility of PPMI Group and do 
not necessarily reflect

## Hierarchical report navigation

Thanks to `toolslm.md_hier` and a clean markdown structure of a `report` markdown, we can create a nested dictionary of section, subsection, ... as follows:

In [ ]:
#| eval: false
hdgs = create_heading_dict(report); hdgs

{'PPMi .... page 1': {},
 'CONTENTS .... page 3': {},
 '1. Introduction .... page 4': {},
 '2. Background of the JI-HoA .... page 5': {'2.1. Context and design of the JI-HoA .... page 5': {},
  '2.2. External factors affecting the implementation of the JI .... page 7': {}},
 '3. Methodology .... page 8': {},
 '4. Findings .... page 10': {'4.1. Relevance .... page 10': {'4.1.1. Relevance of programme activities for migrants, returnees, and communities .... page 10': {}},
  'Overall performance score for relevance: $3.9 / 5$ <br> Robustness score for the evidence: $4.5 / 5$': {'4.1.1.1 Needs of migrants .... page 10': {},
   '4.1.1.2 Needs of returnees .... page 10': {},
   '4.1.1.3 Needs of community members .... page 12': {},
   "4.1.2. Programme's relevance to the needs of stakeholders .... page 12": {'4.1.2.1 Needs of governments .... page 12': {},
    '4.1.2.2 Needs of other stakeholders .... page 13': {}},
   '4.2. Coherence .... page 13': {"4.2.1. The JI-HoA's alignment with the o

In [ ]:
#| exports
def find_section_path(
    hdgs: dict, # The nested dictionary structure
    target_section: str # The section name to find
):
    "Find the nested key path for a given section name"
    def search_recursive(current_dict, path=[]):
        for key, value in current_dict.items():
            current_path = path + [key]
            if key == target_section:
                return current_path
            if isinstance(value, dict):
                result = search_recursive(value, current_path)
                if result:
                    return result
        return None
    
    return search_recursive(hdgs)

Then we can retrieve the subsection path (list of nested headings to reach this specific section) in this nested `hdgs` dict :

In [ ]:
#| eval: false
path = find_section_path(hdgs, "4.1.1.1 Needs of migrants .... page 10"); path

['4. Findings .... page 10',
 'Overall performance score for relevance: $3.9 / 5$ <br> Robustness score for the evidence: $4.5 / 5$',
 '4.1.1.1 Needs of migrants .... page 10']

Then retrieve the specific subsection content:

In [ ]:
#| exports
def get_content_tool(hdgs, keys_list):
    "Navigate through nested levels using the exact key strings"
    return reduce(lambda current, key: current[key], keys_list, hdgs).text

In [ ]:
#| eval: false
content = get_content_tool(hdgs, path)
print(content[:500])

##### 4.1.1.1 Needs of migrants .... page 10

Desk research and interviews confirm that the programme responded to the most pressing needs of migrants. The 
JI-HoA enabled them to return from dangerous environments, such as detention, where no other support was available.
Migrants shared that they suffered on their irregular migration journeys, had acutely distressing experiences ${ 
}^{13}$, and highlighted that their families and communities could not help them ${ }^{14}$. Stakeholders supportin

## Eval framework utils

Utility functions for:

1. Pretty printing of evaluation framework data with truncation
2. Easy attribute-style access to evaluation framework dictionaries

In [ ]:
#| exports
class EvalDict(AttrDict):
    "A dictionary that truncates strings and lists to a maximum length for pretty printing and dot notation access of attributes"
    def __init__(self, *args, max_len=50, max_items=2, **kwargs):
        super().__init__(*args, **kwargs)
        store_attr('max_len,max_items')
    
    def __repr__(self):
        def truncate(text, max_len=self.max_len):
            return text[:max_len] + "..." if len(text) > self.max_len else text
        
        def format_item(key, value):
            if isinstance(value, str):
                return f"'{key}': '{truncate(value)}'"
            elif isinstance(value, (list, dict)):
                count = len(value)
                return f"'{key}': [{count} items]"
            else:
                return f"'{key}': {value}"
        
        items = list(self.items())[:self.max_items]  
        formatted = [format_item(k, v) for k, v in items]
        suffix = ", ..." if len(self) > self.max_items else ""
        
        return f"EvalDict({{{', '.join(formatted)}{suffix}}})"

In [ ]:
#| exports
@delegates(EvalDict.__init__)
def evalify(
    data, # data to evalify
    **kwargs # kwargs to pass to EvalDict.__init__
    ):
    "Evalify a dictionary or a list of dictionaries"
    if isinstance(data, list):
        return [EvalDict(item, **kwargs) for item in data]
    else:
        return EvalDict(data, **kwargs)

For instance on the SRF Enablers and GCM to SRF lookup tables:

In [ ]:
#| eval: false
file_path = Path('files/eval_frameworks/srf_enablers.json')
srf_e = json.loads(file_path.read_text())
srf_e = evalify(srf_e, max_len=40, max_items=5); srf_e

[EvalDict({'id': '1', 'title': 'Workforce', 'description': 'IOM’s diverse and capable people are our...', 'indicators': [14 items], 'max_len': 40, ...}),
 EvalDict({'id': '2', 'title': 'Partnership', 'description': 'Long-term partnerships built on trust me...', 'indicators': [20 items], 'max_len': 40, ...}),
 EvalDict({'id': '3', 'title': 'Funding', 'description': 'IOM’s vision will only be realized with ...', 'indicators': [22 items], 'max_len': 40, ...}),
 EvalDict({'id': '4', 'title': 'Data and evidence', 'description': 'IOM will be the pre-eminent source of mi...', 'indicators': [16 items], 'max_len': 40, ...}),
 EvalDict({'id': '5', 'title': 'Learning and Innovation', 'description': 'As an innovator within the migration spa...', 'indicators': [7 items], 'max_len': 40, ...}),
 EvalDict({'id': '6', 'title': 'Communication', 'description': 'We will take a data-driven and co-design...', 'indicators': [4 items], 'max_len': 40, ...}),
 EvalDict({'id': '7', 'title': 'Internal systems', '

Making access to specific attributes "dotted":

In [ ]:
#| eval: false
srf_e[0].description

'IOM’s diverse and capable people are our most valued asset. Through investing in better workforce planning and people management, we will facilitate their professional development and improve their daily workplace experience. IOM will have flexible systems and procedures in place to ensure it can adapt to the future of work while ensuring the safety, security and well-being of its staff. IOM will strengthen its security posture and foster an inclusive and enabling work environment. Our leadership will have the ability to connect, motivate and inspire a sense of our shared vision and values.'

## Signatures

A [DSPy signature](https://dspy.ai/learn/programming/signatures) is a declarative specification of input/output behavior of a DSPy module. Signatures allow you to tell the LM (Language Model) what it needs to do, rather than specify how we should ask the LM to do it.

### Stage 1

In [ ]:
#| exports
class EnablersCrossCuttingOverview(dspy.Signature):
    """Analyze if report covers enablers or cross-cutting priorities"""
    report_headings: str = dspy.InputField(desc="Document structure")
    category_title: str = dspy.InputField(desc="Category title")
    category_description: str = dspy.InputField(desc="Category description")
    
    relevant_sections: List[str] = dspy.OutputField(desc="Potentially relevant sections")
    confidence: float = dspy.OutputField(desc="Confidence 0-1")
    reasoning: str = dspy.OutputField(desc="Why this confidence level")
    next_section: str = dspy.OutputField(desc="Next section to explore, or 'DONE'")

For instance on "Data and evidence" SRF Enabler number 4:

In [ ]:
#| eval: false
lm = dspy.LM(cfg.lm, api_key=cfg.api_key)
dspy.configure(lm=lm)

overview_analyzer = dspy.ChainOfThought(EnablersCrossCuttingOverview)
data_evidence = srf_e[3]  # "Data and evidence" is at index 3
print(f"Analyzing {data_evidence.title}...")
print(f"{data_evidence.description}\n")

Analyzing Data and evidence...

IOM will be the pre-eminent source of migration and displacement data for action, which help save lives and deliver
solutions; data for insight, which help facilitate regular migration pathways; and data for foresight, which help 
drive anticipatory action. IOM will have the systems and data fluency to collect, safely store, analyze, share and 
apply disaggregated data and evidence across the mobility spectrum. Our extensive data and research repositories 
will underpin evidence-based policies and practices. Data will be central to the internal decision-making and 
management of the Organization.

In [ ]:
#| eval: false
result = overview_analyzer(
    report_headings=str(hdgs),
    category_title=data_evidence.title,
    category_description=data_evidence.description
)

print("Relevant sections:", result.relevant_sections)
print("Confidence:", result.confidence)
print("Reasoning:", result.reasoning)
print("Next section:", result.next_section)

Relevant sections:
['4.1. Relevance', '4.3. Effectiveness', '4.4. Efficiency', '5. Conclusions and Recommendations']

Confidence: 0.7

Reasoning: The category description focuses on data collection, analysis, and application for evidence-based 
policies and practices. Several sections in the report discuss data availability, evidence robustness, and the 
achievement of objectives based on data. Therefore, these sections are potentially relevant.

Next section: 4.5. Sustainability

In [ ]:
#| exports
class EnablersCrossCuttingExploration(dspy.Signature):
    """Explore specific report sections for enablers/cross-cutting evidence"""
    section_content: str = dspy.InputField(desc="Content of current section")
    category_title: str = dspy.InputField(desc="Category title")
    category_description: str = dspy.InputField(desc="Category description")
    evidence_so_far: str = dspy.InputField(desc="Evidence collected from previous sections")
    
    evidence_found: str = dspy.OutputField(desc="New evidence found in this section")
    confidence: float = dspy.OutputField(desc="Updated confidence 0-1")
    reasoning: str = dspy.OutputField(desc="Why this confidence level")
    next_action: str = dspy.OutputField(desc="'EXPLORE:section_name' or 'DONE'")

In [ ]:
#| eval: false
condensed_gcm = {
    "7": {
        "title": "Address and reduce vulnerabilities in migration",
        "core_theme": "Protect migrants in vulnerable situations through comprehensive support and rights-based approaches",
        "key_principles": ["Human rights-based approach", "Best interests of the child", "Gender-responsive policies", "Non-discrimination"],
        "target_groups": ["Unaccompanied children", "Women at risk", "Trafficking victims", "Workers facing exploitation", "Persons with disabilities"],
        "main_activities": ["Identification and assistance", "Legal protection and remedies", "Child protection systems", "Status regularization procedures", "Crisis response"]
    },
    "21": {
        "title": "Cooperate in facilitating safe and dignified return and readmission, as well as sustainable reintegration",
        "core_theme": "Safe and dignified return, readmission, and sustainable reintegration of migrants",
        "key_principles": ["Due process and individual assessment", "Prohibition of collective expulsion", "Non-refoulement", "Human right to return"],
        "target_groups": ["Returning migrants", "Children in return processes", "Communities of origin"],
        "main_activities": ["Cooperation frameworks", "Travel documents and identification", "Consular assistance", "Reintegration support", "Monitoring mechanisms"]
    }
}